# Le modèle du Brusselator

Le modèle du Brusselator est un exemple de modèle de réaction chimique autocatalytique. Il s'écrit
sous la forme du système suivant:

$$
\left\{\begin{aligned}
\partial_t y_1 & = 1 - (b+1) y_1 + a y_1^2y_2\\
\partial_t y_2 & = b y_1 - a y_1^2y_2
\end{aligned}\right.
$$

In [ ]:
import numpy as np
import time

from scipy.integrate import solve_ivp

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import column

from mylib.model import brusselator_model
import mylib.integration as integration
from mylib.integration import parareal

output_notebook(hide_banner=True)

In [ ]:
def compute_integration_time(tini, tend, yini, fcn, method, nt, rtol, atol, ntimes=100):
        
    t0 = time.time()
    for i in range(ntimes):
        integration.integrate(method, tini, tend, nt, rtol, atol, yini, fcn)
    t1 = time.time()
    time_average = (t1-t0) / ntimes
    
    return time_average

In [ ]:
def plot_sol(tini, tend, nb_sub, sol_ref, sol):

    # plot solution
    fig_sol = figure(x_range=(tini-0.25, tend+0.25), width=950, height=400, 
                     title="Solution (click on legend to see reference solution)")
    # plot ref solution
    for isub in range(nb_sub):
        fig_sol.line(sol_ref.sol_fine[isub].t, sol_ref.sol_fine[isub].y[0], color="grey", line_alpha=0.5)
        fig_sol.line(sol_ref.sol_fine[isub].t, sol_ref.sol_fine[isub].y[1], color="grey", line_alpha=0.5)
        
    # plot Y
    fig_sol.x(sol.T, sol.Y[0], line_width=2, size=10)    
    fig_sol.x(sol.T, sol.Y[1], line_width=2, size=10, color="green")
    
    #plot fine solution
    for isub in range(nb_sub):
        fig_sol.line(sol.sol_fine[isub].t, sol.sol_fine[isub].y[0], legend='y1')
        fig_sol.line(sol.sol_fine[isub].t, sol.sol_fine[isub].y[1], legend='y2', color="green")

    fig_sol.legend.location = "top_left"
    fig_sol.legend.click_policy="hide"

    show(fig_sol)

## Solution obtenue par l'algorithme pararéel 

In [ ]:
def parareal_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    # parareal parameters
    nb_sub = 100
    coarse_method = "rk1"
    nc = 10
    rtolc = 1.e-6
    atolc = 1.e-6
    fine_method = "rk4"
    nf = 100
    rtolf = 1.e-6
    atolf = 1.e-6
    max_iter = nb_sub
    eps = 1.e-5
    
    # compute time for coarse integration
    Tg = compute_integration_time(tini, tend, yini, fcn, coarse_method, nb_sub*nc, rtolc, atolc, ntimes=100)
    print(f"Elapsed time (s) for coarse integration : {Tg:.8f}")
    # compute time for fine integration
    Tf = compute_integration_time(tini, tend, yini, fcn, fine_method, nb_sub*nf, rtolf, atolf, ntimes=10)
    print(f"Elapsed time (s) for fine integration   : {Tf:.8f}")
    
    # compute reference solution
    print("\nCompute reference solution ...")
    sol_ref = integration.compute_ref_sol(tini, tend, nb_sub, yini, fcn, 'RK45')
    
    # parareal algorithm
    print("\nParareal algorithm\n")     
    sol = parareal(tini, tend, nb_sub, yini, fcn, coarse_method, nc, rtolc, atolc, fine_method, nf, rtolf, atolf,
                   max_iter, eps, np.ones(2))

    print("\nGlobal error :")     
    err = np.zeros(2)
    for ieq in range(2):
        err[ieq] = np.linalg.norm(sol.Y[ieq]-sol_ref.Y[ieq])/np.sqrt(nb_sub)
        print(f" {ieq+1}th variable : || Yi - Yref || = {err[ieq]:7.2e} (error between parareal and ref solution)")

    print("\nEfficiency :")
    print(f"  nb iteration (nit) = {sol.nit}")
    print(f"  nit/n = {sol.nit/nb_sub}")
    print(f"  Tg/Tf = {Tg/Tf}")
    Tpar = (sol.nit/nb_sub) + (sol.nit+1)*(Tg/Tf) 
    print(f"  Tseq/Tpar = {1/Tpar}")

    # plot solution
    plot_sol(tini, tend, nb_sub, sol_ref, sol)

parareal_sol()